# MMAD MCQ Evaluation - Qwen3-VL-8B

기존 `eval_llm_baseline.py` 파이프라인 **그대로** 사용, 모델만 Qwen3-VL-8B로 교체

**사전 설정:** 런타임 → 런타임 유형 변경 → **A100 GPU**

## 1. GPU 확인

In [ ]:
!nvidia-smi
import torch
print(f"\nCUDA: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")


Sat Feb  7 03:28:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2. 패키지 설치

In [ ]:
!pip install -q transformers>=4.51.0 accelerate qwen-vl-utils torchvision
!pip install -q tqdm pillow

## 3. Google Drive 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!du -sh "/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD"

0	/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD


In [20]:
import json, os

src_root = "/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD"
dst_root = "/content/MMAD"
json_path = os.path.join(src_root, "mmad_10classes.json")

with open(json_path, "r") as f:
    data = json.load(f)

# 파일 목록 수집
all_files = set()
for img_path, meta in data.items():
    all_files.add(img_path)
    for t in meta.get("random_templates", []):
        all_files.add(t)

print(f"복사할 파일: {len(all_files)}개")

# 디렉토리 생성
dirs = set(os.path.dirname(os.path.join(dst_root, f)) for f in all_files)
for d in dirs:
    os.makedirs(d, exist_ok=True)

# 파일 목록 저장
with open("/content/copy_list.txt", "w") as f:
    for p in all_files:
        f.write(p + "\n")

# JSON 복사
os.makedirs(dst_root, exist_ok=True)
!cp "{json_path}" "{dst_root}/mmad_10classes.json"

# rsync로 복사 (진행도 표시)
!rsync -a --info=progress2 --files-from=/content/copy_list.txt "{src_root}/" "{dst_root}/"

!echo "✅ 완료!"
!find /content/MMAD -type f | wc -l

복사할 파일: 8773개
 15,237,129,298  79%    2.86MB/s    1:24:49 (xfr#7246, to-chk=0/36764)
✅ 완료!
8774


## 4. 경로 설정 & 프로젝트 import

In [8]:
import os, sys

# ============================================================
# ★ 경로 설정 ★
# ============================================================
PROJECT_ROOT = "/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation"
DATA_ROOT = os.path.join(PROJECT_ROOT, "dataset", "MMAD")
MMAD_JSON = os.path.join(DATA_ROOT, "/content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD/mmad_10classes.json")  # ← json 위치에 맞게 수정
# ============================================================

# 프로젝트를 import path에 추가
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

assert os.path.exists(PROJECT_ROOT), f"❌ PROJECT_ROOT: {PROJECT_ROOT}"
assert os.path.exists(DATA_ROOT), f"❌ DATA_ROOT: {DATA_ROOT}"
assert os.path.exists(MMAD_JSON), f"❌ MMAD_JSON: {MMAD_JSON}"

print(f"✅ 프로젝트: {PROJECT_ROOT}")
print(f"✅ 데이터: {DATA_ROOT}")
print(f"✅ JSON: {MMAD_JSON}")

✅ 프로젝트: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation
✅ 데이터: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD
✅ JSON: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD/mmad_10classes.json


## 5. qwen_client.py 패치 (Qwen3-VL 지원 추가)

기존 코드를 직접 수정하지 않고, 런타임에서 `_load_model`만 패치합니다.

In [9]:
from src.mllm.qwen_client import QwenVLClient

# 원본 _load_model 백업
_original_load_model = QwenVLClient._load_model

def _patched_load_model(self):
    """Qwen3-VL / Qwen2.5-VL 자동 분기 패치"""
    if self._model is not None:
        return

    from transformers import AutoProcessor

    attn_impl = "flash_attention_2" if self._use_flash_attention else "eager"

    # 모델명에 따라 자동 분기
    if "qwen3" in self.model_path.lower():
        from transformers import Qwen3VLForConditionalGeneration as ModelClass
        print(f"🔵 Qwen3-VL 모드: {self.model_path}")
    else:
        from transformers import Qwen2_5_VLForConditionalGeneration as ModelClass
        print(f"🟢 Qwen2.5-VL 모드: {self.model_path}")

    try:
        self._model = ModelClass.from_pretrained(
            self.model_path,
            torch_dtype=self.torch_dtype,
            attn_implementation=attn_impl,
            device_map=self.device,
        )
        print(f"✅ 모델 로드 완료 (attn: {attn_impl})")
    except Exception as e:
        print(f"⚠️ {attn_impl} 실패, eager로 fallback: {e}")
        self._model = ModelClass.from_pretrained(
            self.model_path,
            torch_dtype=self.torch_dtype,
            attn_implementation="eager",
            device_map=self.device,
        )

    self._processor = AutoProcessor.from_pretrained(
        self.model_path,
        min_pixels=self.min_pixels,
        max_pixels=self.max_pixels,
    )

    print(f"✅ GPU 메모리: {torch.cuda.memory_allocated()/1e9:.1f} GB")

# 패치 적용
import torch
QwenVLClient._load_model = _patched_load_model
print("✅ QwenVLClient 패치 완료 - Qwen3-VL 지원 활성화")

✅ QwenVLClient 패치 완료 - Qwen3-VL 지원 활성화


## 6. MODEL_REGISTRY에 Qwen3-VL 추가 & eval 스크립트 패치

In [10]:
# eval 스크립트의 레지스트리도 런타임 패치
# (스크립트를 직접 실행하지 않고 함수를 import해서 쓰므로)

# 방법: eval 스크립트를 모듈로 import하고 레지스트리 수정
sys.argv = ['eval']  # argparse 방지

import importlib
spec = importlib.util.spec_from_file_location(
    "eval_llm_baseline",
    os.path.join(PROJECT_ROOT, "scripts", "eval_llm_baseline.py")
)
eval_module = importlib.util.module_from_spec(spec)

# MODEL_REGISTRY 패치를 위해 먼저 로드
try:
    spec.loader.exec_module(eval_module)
except SystemExit:
    pass  # argparse가 exit할 수 있음

print("✅ eval 모듈 로드 완료")

✅ eval 모듈 로드 완료


## 7. 평가 실행

기존 `eval_llm_baseline.py`를 **커맨드라인 그대로** 실행합니다.

In [11]:
# eval_llm_baseline.py에 Qwen3-VL-8B 레지스트리 추가 (파일 직접 수정)
eval_script = os.path.join(PROJECT_ROOT, "scripts", "eval_llm_baseline.py")

with open(eval_script, "r", encoding="utf-8") as f:
    content = f.read()

# 이미 추가되어 있는지 확인
if '"qwen3-vl-8b"' not in content:
    # qwen2-vl 라인 뒤에 추가
    old_line = '    "qwen2-vl": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen2-VL-7B-Instruct"},'
    new_lines = old_line + '\n    "qwen3-vl-8b": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen3-VL-8B"},'
    content = content.replace(old_line, new_lines)

    with open(eval_script, "w", encoding="utf-8") as f:
        f.write(content)
    print("✅ eval_llm_baseline.py에 qwen3-vl-8b 레지스트리 추가")
else:
    print("✅ qwen3-vl-8b 이미 등록됨")

✅ qwen3-vl-8b 이미 등록됨


In [12]:
# qwen_client.py에 Qwen3-VL 분기 코드 영구 적용
qwen_client_path = os.path.join(PROJECT_ROOT, "src", "mllm", "qwen_client.py")

with open(qwen_client_path, "r", encoding="utf-8") as f:
    content = f.read()

# 이미 패치되어 있는지 확인
if 'Qwen3VLForConditionalGeneration' not in content:
    # _load_model 내부의 import 부분을 교체
    old_import = '        from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor'
    new_import = """        from transformers import AutoProcessor

        # Qwen3-VL vs Qwen2.5-VL 자동 분기
        if "qwen3" in self.model_path.lower():
            from transformers import Qwen3VLForConditionalGeneration as Qwen2_5_VLForConditionalGeneration
        else:
            from transformers import Qwen2_5_VLForConditionalGeneration"""

    content = content.replace(old_import, new_import)

    with open(qwen_client_path, "w", encoding="utf-8") as f:
        f.write(content)
    print("✅ qwen_client.py 패치 완료 - Qwen3-VL 자동 분기 추가")
else:
    print("✅ qwen_client.py 이미 패치됨")

✅ qwen_client.py 이미 패치됨


In [13]:
# 패치 확인
print("=== eval_llm_baseline.py - qwen 관련 레지스트리 ===")
with open(os.path.join(PROJECT_ROOT, "scripts", "eval_llm_baseline.py"), "r") as f:
    for line in f:
        if 'qwen' in line.lower() and 'registry' not in line.lower() and '"type"' in line:
            print(line.rstrip())

print("\n=== qwen_client.py - _load_model import 부분 ===")
with open(os.path.join(PROJECT_ROOT, "src", "mllm", "qwen_client.py"), "r") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        if 'Qwen' in line and ('import' in line or 'qwen3' in line.lower()):
            print(f"  L{i+1}: {line.rstrip()}")

=== eval_llm_baseline.py - qwen 관련 레지스트리 ===
    "qwen": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen2.5-VL-7B-Instruct"},
    "qwen-7b": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen2.5-VL-7B-Instruct"},
    "qwen-2b": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen2.5-VL-2B-Instruct"},
    "qwen2-vl": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen2-VL-7B-Instruct"},
    "qwen3-vl-8b": {"type": "local", "class": "QwenVLClient", "model": "Qwen/Qwen3-VL-8B-Instruct"},
    "llava-onevision": {"type": "local", "class": "LLaVAClient", "model": "llava-hf/llava-onevision-qwen2-7b-ov-hf"},

=== qwen_client.py - _load_model import 부분 ===
  L56:         # Qwen3-VL vs Qwen2.5-VL 자동 분기
  L58:             from transformers import Qwen3VLForConditionalGeneration as ModelClass
  L60:             from transformers import Qwen2_5_VLForConditionalGeneration as ModelClass


## 8. 평가 실행 🚀

기존 스크립트를 커맨드라인 그대로 실행합니다.

In [18]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 61.4 MB/s eta 0:00:00


In [14]:
from huggingface_hub import login
login(token="YOUR_HF_TOKEN")

In [19]:
# 먼저 10장으로 테스트
!cd {PROJECT_ROOT} && python scripts/eval_llm_baseline.py \
    --model qwen3-vl-8b \
    --few-shot 1 \
    --batch-mode \
    --data-root "{DATA_ROOT}" \
    --mmad-json "{MMAD_JSON}" \
    --max-images 10 \
    --output-dir output/eval

MMAD LLM Evaluation
Model: qwen3-vl-8b
Few-shot: 1
Template: Random_template
With AD: False
Data root: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD
MMAD JSON: /content/drive/Othercomputers/mynotebook/multimodal-anomaly-report-generation/dataset/MMAD/mmad_10classes.json
Output: output/eval/answers_1_shot_qwen3-vl-8b_Random_template.json

Total images: 10

Evaluating:   0%|                                                            | 0/10 [00:00<?, ?it/s]

Fetching 4 files: 100% 4/4 [00:00<00:00, 9868.95it/s]

Download complete: : 0.00B [00:00, ?B/s]              



Fetching 4 files: 100% 4/4 [00:00<00:00, 6756.83it/s]


Download complete: : 0.00B [00:00, ?B/s]
Download complete: : 0.00B [00:01, ?B/s]

Loading weights:   0% 0/750 [00:00<?, ?it/s]
Loading weights:   0% 1/750 [00:00<00:00, 11155.06it/s, Materializing param=lm_head.weight]
Loading weights:   0% 1/750 [00:00<00:00, 6432.98it/s, Materializing param=lm_head.weight] 
Loading weight

In [21]:
!cd {PROJECT_ROOT} && python scripts/eval_llm_baseline.py \
    --model qwen3-vl-8b \
    --few-shot 1 \
    --batch-mode \
    --data-root "/content/MMAD" \
    --mmad-json "/content/MMAD/mmad_10classes.json" \
    --max-images 10 \
    --output-dir output/eval

MMAD LLM Evaluation
Model: qwen3-vl-8b
Few-shot: 1
Template: Random_template
With AD: False
Data root: /content/MMAD
MMAD JSON: /content/MMAD/mmad_10classes.json
Output: output/eval/answers_1_shot_qwen3-vl-8b_Random_template.json

Total images: 10

Evaluating:   0%|                                                            | 0/10 [00:00<?, ?it/s]

Fetching 4 files: 100% 4/4 [00:00<00:00, 13443.28it/s]

Download complete: : 0.00B [00:00, ?B/s]              



Fetching 4 files: 100% 4/4 [00:00<00:00, 8603.70it/s]


Download complete: : 0.00B [00:01, ?B/s]
Download complete: : 0.00B [00:00, ?B/s]

Loading weights:   0% 0/750 [00:00<?, ?it/s]
Loading weights:   0% 1/750 [00:00<00:00, 10727.12it/s, Materializing param=lm_head.weight]
Loading weights:   0% 1/750 [00:00<00:00, 5849.80it/s, Materializing param=lm_head.weight] 
Loading weights:   0% 2/750 [00:00<01:26,  8.66it/s, Materializing param=lm_head.weight]  
Loading weights:   0% 2/750 [00:00<01:26,  8.66it/s, Materializing param=mod

In [22]:
import json
from collections import defaultdict

with open("/content/MMAD/mmad_10classes.json", "r") as f:
    data = json.load(f)

# 데이터셋/클래스/유형별 분류
# 예: MVTec-LOCO/breakfast_box/test/good/000.png → MVTec-LOCO/breakfast_box/good
# 예: MVTec-LOCO/breakfast_box/test/structural_anomalies/001.png → MVTec-LOCO/breakfast_box/structural_anomalies

structure = defaultdict(list)
for img_path in data.keys():
    parts = img_path.split("/")
    # parts: [dataset, class, test, defect_type, filename]
    dataset_class = f"{parts[0]}/{parts[1]}"
    defect_type = parts[3]  # good, bad, structural_anomalies 등
    key = f"{dataset_class}/{defect_type}"
    structure[key].append(img_path)

print(f"총 폴더(유형) 수: {len(structure)}개 → 각 1장씩 = {len(structure)}장\n")

for key in sorted(structure.keys()):
    print(f"  {key}: {len(structure[key])}장")

총 폴더(유형) 수: 33개 → 각 1장씩 = 33장

  GoodsAD/cigarette_box/good: 183장
  GoodsAD/cigarette_box/opened: 246장
  GoodsAD/drink_bottle/cap_half_open: 73장
  GoodsAD/drink_bottle/cap_open: 79장
  GoodsAD/drink_bottle/good: 350장
  GoodsAD/drink_bottle/surface_damage: 273장
  GoodsAD/drink_can/deformation: 61장
  GoodsAD/drink_can/good: 145장
  GoodsAD/drink_can/straw_missing: 24장
  GoodsAD/drink_can/surface_damage: 61장
  GoodsAD/food_bottle/deformation: 86장
  GoodsAD/food_bottle/good: 235장
  GoodsAD/food_bottle/opened: 119장
  GoodsAD/food_bottle/surface_damage: 156장
  GoodsAD/food_box/deformation: 76장
  GoodsAD/food_box/good: 144장
  GoodsAD/food_box/opened: 101장
  GoodsAD/food_box/surface_damage: 74장
  GoodsAD/food_package/broken: 86장
  GoodsAD/food_package/good: 252장
  GoodsAD/food_package/surface_anomaly: 144장
  MVTec-LOCO/breakfast_box/good: 102장
  MVTec-LOCO/breakfast_box/logical_anomalies: 83장
  MVTec-LOCO/breakfast_box/structural_anomalies: 90장
  MVTec-LOCO/juice_bottle/good: 94장
  MVTec-LOCO/ju

## 33개 폴더 3장 씩 평가

In [23]:
import json, random

with open("/content/MMAD/mmad_10classes.json", "r") as f:
    data = json.load(f)

# 폴더별 분류
from collections import defaultdict
structure = defaultdict(list)
for img_path in data.keys():
    parts = img_path.split("/")
    key = f"{parts[0]}/{parts[1]}/{parts[3]}"
    structure[key].append(img_path)

# 각 폴더에서 N장씩 샘플링
N = 3  # 1, 3, 5 등 조절 가능
random.seed(42)

sampled = []
for key in sorted(structure.keys()):
    imgs = structure[key]
    sampled += random.sample(imgs, min(N, len(imgs)))

# 저장
sample_data = {k: data[k] for k in sampled}
sample_path = "/content/MMAD/mmad_sample_99.json"
with open(sample_path, "w") as f:
    json.dump(sample_data, f, indent=2, ensure_ascii=False)

n_good = sum(1 for s in sample_data if "/good/" in s)
n_bad = len(sample_data) - n_good
print(f"✅ {len(sample_data)}장 (normal={n_good}, anomaly={n_bad})")
print(f"   33폴더 × {N}장씩")
print(f"   저장: {sample_path}")

✅ 99장 (normal=30, anomaly=69)
   33폴더 × 3장씩
   저장: /content/MMAD/mmad_sample_99.json


In [25]:
# eval 실행 전에 이 셀 실행
from src.mllm.qwen_client import QwenVLClient

# 원본 __init__ 백업
_original_init = QwenVLClient.__init__

def _patched_init(self, *args, **kwargs):
    kwargs['min_pixels'] = 700 * 700  # 490,000
    kwargs['max_pixels'] = 700 * 700  # 490,000 (고정)
    _original_init(self, *args, **kwargs)

QwenVLClient.__init__ = _patched_init
print("✅ 이미지 크기 700x700 고정")

✅ 이미지 크기 700x700 고정


In [27]:
qwen_path = f"{PROJECT_ROOT}/src/mllm/qwen_client.py"

with open(qwen_path, "r") as f:
    content = f.read()

content = content.replace(
    "min_pixels: int = 64 * 28 * 28",
    "min_pixels: int = 700 * 700"
).replace(
    "max_pixels: int = 1280 * 28 * 28",
    "max_pixels: int = 700 * 700"
)

with open(qwen_path, "w") as f:
    f.write(content)

print("✅ qwen_client.py 수정 완료")

✅ qwen_client.py 수정 완료


In [28]:
!cd {PROJECT_ROOT} && python scripts/eval_llm_baseline.py \
    --model qwen3-vl-8b \
    --few-shot 1 \
    --batch-mode \
    --data-root "/content/MMAD" \
    --mmad-json "/content/MMAD/mmad_sample_99.json" \
    --output-dir output/eval

MMAD LLM Evaluation
Model: qwen3-vl-8b
Few-shot: 1
Template: Random_template
With AD: False
Data root: /content/MMAD
MMAD JSON: /content/MMAD/mmad_sample_99.json
Output: output/eval/answers_1_shot_qwen3-vl-8b_Random_template.json

Total images: 99

Evaluating:   0%|                                                            | 0/99 [00:00<?, ?it/s]

Fetching 4 files: 100% 4/4 [00:00<00:00, 13107.20it/s]

Download complete: : 0.00B [00:00, ?B/s]              



Fetching 4 files: 100% 4/4 [00:00<00:00, 61008.06it/s]


Download complete: : 0.00B [00:00, ?B/s]
Download complete: : 0.00B [00:01, ?B/s]

Loading weights:   0% 0/750 [00:00<?, ?it/s]
Loading weights:   0% 1/750 [00:00<00:00, 11983.73it/s, Materializing param=lm_head.weight]
Loading weights:   0% 1/750 [00:00<00:00, 6626.07it/s, Materializing param=lm_head.weight] 
Loading weights:   0% 2/750 [00:00<01:28,  8.41it/s, Materializing param=lm_head.weight]  
Loading weights:   0% 2/750 [00:00<01:28,  8.41it/s, Materializing param=mo

In [30]:
!cd {PROJECT_ROOT} && python scripts/eval_llm_baseline.py \
    --model llava \
    --few-shot 1 \
    --data-root "/content/MMAD" \
    --mmad-json "/content/MMAD/mmad_sample_99.json" \
    --output-dir output/eval

MMAD LLM Evaluation
Model: llava
Few-shot: 1
Template: Random_template
With AD: False
Data root: /content/MMAD
MMAD JSON: /content/MMAD/mmad_sample_99.json
Output: output/eval/answers_1_shot_llava_Random_template.json

Total images: 99

Evaluating:   0%|                                                            | 0/99 [00:00<?, ?it/s]

Fetching 3 files: 100% 3/3 [00:00<00:00, 12360.42it/s]

Download complete: : 0.00B [00:00, ?B/s]

Loading weights:   0% 0/686 [00:00<?, ?it/s]
Loading weights:   0% 1/686 [00:00<00:00, 12052.60it/s, Materializing param=lm_head.weight]
Loading weights:   0% 1/686 [00:00<00:00, 6574.14it/s, Materializing param=lm_head.weight] 
Loading weights:   0% 2/686 [00:00<00:12, 54.46it/s, Materializing param=model.language_model.embed_tokens.weight]
Loading weights:   0% 2/686 [00:00<00:12, 54.23it/s, Materializing param=model.language_model.embed_tokens.weight]
Loading weights:   0% 3/686 [00:00<00:08, 80.93it/s, Materializing param=model.language_model.layers.0.i

In [ ]:
# ★ 전체 평가 실행 ★
!cd {PROJECT_ROOT} && python scripts/eval_llm_baseline.py \
    --model qwen3-vl-8b \
    --few-shot 1 \
    --similar-template \
    --data-root "{DATA_ROOT}" \
    --mmad-json "{MMAD_JSON}" \
    --output-dir output/eval \
    --resume

## 9. 결과 확인

In [ ]:
import json
from collections import defaultdict

result_path = os.path.join(PROJECT_ROOT, "output", "eval", "answers_1_shot_qwen3-vl-8b_Similar_template.json")

if os.path.exists(result_path):
    with open(result_path, "r") as f:
        results = json.load(f)

    total = len(results)
    correct = sum(1 for r in results if r["correct_answer"] == r["gpt_answer"])
    print(f"{'='*60}")
    print(f"Overall: {correct}/{total} = {correct/total:.1%}")
    print(f"{'='*60}\n")

    # Question Type별
    type_stats = defaultdict(lambda: {"c": 0, "t": 0})
    for r in results:
        qt = r["question_type"]
        type_stats[qt]["t"] += 1
        if r["correct_answer"] == r["gpt_answer"]:
            type_stats[qt]["c"] += 1

    print(f"{'Type':<25} {'Acc':>10}")
    print("-" * 40)
    for qt in sorted(type_stats):
        s = type_stats[qt]
        print(f"{qt:<25} {s['c']}/{s['t']:>4} = {s['c']/s['t']:.1%}")
else:
    print(f"결과 파일 없음: {result_path}")
    print("output/eval/ 내용:")
    !ls -la {PROJECT_ROOT}/output/eval/ 2>/dev/null || echo "디렉토리 없음"